In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
df_main = pd.read_csv('main.csv')
df_address = pd.read_csv('address.csv')
df_paid = pd.read_csv('paid_record.csv')

In [3]:
df_main.head()

,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,DAYS_BIRTH_CLEAN,Letter,Main_ID
0,M,Y,Y,0.0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-4542.0,1.0,1.0,0.0,0.0,NaN,2.0,09/04/1990,L,5008804
1,M,Y,Y,0.0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-4542.0,1.0,1.0,0.0,0.0,NaN,2.0,09/04/1990,G,5008805
2,M,Y,Y,0.0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-1134.0,1.0,0.0,0.0,0.0,Security staff,2.0,06/05/1964,D,5008806
3,F,N,Y,0.0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-3051.0,1.0,0.0,1.0,1.0,Sales staff,1.0,26/10/1970,I,5008808
4,F,NaN,Y,0.0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-3051.0,1.0,0.0,1.0,1.0,Sales staff,1.0,26/10/1970,Z,5008809


In [4]:
df_address.head()

,New_ID,ADDRESS
0,5008804L,"06042, CT, 31 Mitchell Road"
1,5008805G,"72703, AR, 1189 Northwest End Avenue"
2,5008806D,"72701, AR, 218 Shipley Alley"
3,5008808I,"3138 P Street Northwest, 20007, DC"
4,5008809Z,"2405 230th Street, 21122, MD"


In [5]:
df_paid.head()

,ID,MONTHS_BALANCE,STATUS
0,5001711,0,X
1,5001711,-1,0
2,5001711,-2,0
3,5001711,-3,0
4,5001712,0,C


In [6]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438757 entries, 0 to 438756
Data columns (total 19 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   CODE_GENDER          427894 non-null  object 
 1   FLAG_OWN_CAR         427705 non-null  object 
 2   FLAG_OWN_REALTY      427806 non-null  object 
 3   CNT_CHILDREN         427826 non-null  float64
 4   AMT_INCOME_TOTAL     427624 non-null  float64
 5   NAME_INCOME_TYPE     427718 non-null  object 
 6   NAME_EDUCATION_TYPE  427654 non-null  object 
 7   NAME_FAMILY_STATUS   427772 non-null  object 
 8   NAME_HOUSING_TYPE    427693 non-null  object 
 9   DAYS_EMPLOYED        427784 non-null  float64
 10  FLAG_MOBIL           427690 non-null  float64
 11  FLAG_WORK_PHONE      427715 non-null  float64
 12  FLAG_PHONE           427674 non-null  float64
 13  FLAG_EMAIL           427770 non-null  float64
 14  OCCUPATION_TYPE      296952 non-null  object 
 15  CNT_FAM_MEMBERS  

In [7]:
df_main.isna().sum()

CODE_GENDER             10863
FLAG_OWN_CAR            11052
FLAG_OWN_REALTY         10951
CNT_CHILDREN            10931
AMT_INCOME_TOTAL        11133
NAME_INCOME_TYPE        11039
NAME_EDUCATION_TYPE     11103
NAME_FAMILY_STATUS      10985
NAME_HOUSING_TYPE       11064
DAYS_EMPLOYED           10973
FLAG_MOBIL              11067
FLAG_WORK_PHONE         11042
FLAG_PHONE              11083
FLAG_EMAIL              10987
OCCUPATION_TYPE        141805
CNT_FAM_MEMBERS         10934
DAYS_BIRTH_CLEAN        10715
Letter                      0
Main_ID                     0
dtype: int64

In [8]:
df_address.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438757 entries, 0 to 438756
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   New_ID   438757 non-null  object
 1   ADDRESS  438757 non-null  object
dtypes: object(2)
memory usage: 3.3+ MB


In [9]:
df_paid.isna().sum()

ID                0
MONTHS_BALANCE    0
STATUS            0
dtype: int64

In [10]:
df_address.isna().sum()

New_ID     0
ADDRESS    0
dtype: int64

In [11]:
df_main.dtypes

CODE_GENDER             object
FLAG_OWN_CAR            object
FLAG_OWN_REALTY         object
CNT_CHILDREN           float64
AMT_INCOME_TOTAL       float64
NAME_INCOME_TYPE        object
NAME_EDUCATION_TYPE     object
NAME_FAMILY_STATUS      object
NAME_HOUSING_TYPE       object
DAYS_EMPLOYED          float64
FLAG_MOBIL             float64
FLAG_WORK_PHONE        float64
FLAG_PHONE             float64
FLAG_EMAIL             float64
OCCUPATION_TYPE         object
CNT_FAM_MEMBERS        float64
DAYS_BIRTH_CLEAN        object
Letter                  object
Main_ID                  int64
dtype: object

In [12]:
df_main["CODE_GENDER"].fillna('F', inplace=True)
df_main["FLAG_OWN_CAR"].fillna('N', inplace=True)
df_main["FLAG_OWN_REALTY"].fillna('Y', inplace=True)

In [13]:
#change flag Yes/No in Own Car variables to boolean
df_main.loc[df_main['FLAG_OWN_CAR'] == 'Y', 'FLAG_OWN_CAR'] = 1
df_main.loc[df_main['FLAG_OWN_CAR'] == 'N', 'FLAG_OWN_CAR'] = 0
df_main['FLAG_OWN_CAR'] = df_main['FLAG_OWN_CAR'].astype('bool')
df_main['FLAG_OWN_CAR']

0          True
1          True
2          True
3         False
4         False
          ...  
438752    False
438753     True
438754    False
438755    False
438756    False
Name: FLAG_OWN_CAR, Length: 438757, dtype: bool

In [14]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438757 entries, 0 to 438756
Data columns (total 19 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   CODE_GENDER          438757 non-null  object 
 1   FLAG_OWN_CAR         438757 non-null  bool   
 2   FLAG_OWN_REALTY      438757 non-null  object 
 3   CNT_CHILDREN         427826 non-null  float64
 4   AMT_INCOME_TOTAL     427624 non-null  float64
 5   NAME_INCOME_TYPE     427718 non-null  object 
 6   NAME_EDUCATION_TYPE  427654 non-null  object 
 7   NAME_FAMILY_STATUS   427772 non-null  object 
 8   NAME_HOUSING_TYPE    427693 non-null  object 
 9   DAYS_EMPLOYED        427784 non-null  float64
 10  FLAG_MOBIL           427690 non-null  float64
 11  FLAG_WORK_PHONE      427715 non-null  float64
 12  FLAG_PHONE           427674 non-null  float64
 13  FLAG_EMAIL           427770 non-null  float64
 14  OCCUPATION_TYPE      296952 non-null  object 
 15  CNT_FAM_MEMBERS  

In [15]:
#change flag Yes/No in OWN_REALTY  variables to boolean
df_main.loc[df_main['FLAG_OWN_REALTY'] == 'Y', 'FLAG_OWN_REALTY'] = 1
df_main.loc[df_main['FLAG_OWN_REALTY'] == 'N', 'FLAG_OWN_REALTY'] = 0
df_main['FLAG_OWN_REALTY'] = df_main['FLAG_OWN_REALTY'].astype('bool')
df_main['FLAG_OWN_REALTY']

0          True
1          True
2          True
3          True
4          True
          ...  
438752    False
438753     True
438754     True
438755     True
438756     True
Name: FLAG_OWN_REALTY, Length: 438757, dtype: bool

In [16]:
df_main.FLAG_OWN_REALTY.unique()

array([ True, False])

In [17]:

del(df_main["FLAG_MOBIL"])

def obtener_lista_variables(dataset):
    lista_numericas=[]
    lista_boolean=[]
    lista_categoricas=[]

    for i in dataset:
        if    (dataset[i].dtype.kind in ("f","i")):
              lista_numericas.append(i)
        elif  (dataset[i].dtype.kind in ("f","i", "O")) and len(dataset[i].unique())== 2:
              lista_boolean.append(i)
        elif  (dataset[i].dtype.kind == "O"):
              lista_categoricas.append(i)
                
    return lista_numericas, lista_boolean, lista_categoricas
lista_numericas, lista_boolean, lista_categoricas= obtener_lista_variables(df_main)

for i in lista_numericas:
    median_columna = df_main[i].median()
    df_main[i].fillna(median_columna, inplace=True)
    
df_main["NAME_INCOME_TYPE"].fillna('Working', inplace=True)
df_main["NAME_EDUCATION_TYPE"].fillna('Secondary / secondary special', inplace=True)
df_main["NAME_FAMILY_STATUS"].fillna('Married', inplace=True)
df_main["NAME_HOUSING_TYPE"].fillna('House / apartment', inplace=True)
df_main["OCCUPATION_TYPE"].fillna('Laborers', inplace=True)

In [18]:
#cast DAYS_BIRTH_CLEAN as datetime
df_main['BIRTH_YEAR'] = pd.to_datetime(df_main['DAYS_BIRTH_CLEAN'], format='%d/%m/%Y').dt.year

In [19]:
df_main['AGE'] = 2023 - df_main['BIRTH_YEAR']

In [20]:
df_main['AGE']

0         33.0
1         33.0
2         59.0
3         53.0
4         53.0
          ... 
438752    32.0
438753    45.0
438754    49.0
438755    58.0
438756    34.0
Name: AGE, Length: 438757, dtype: float64

In [21]:
df_main.loc[df_main['DAYS_EMPLOYED'] < 0, 'DAYS_EMPLOYED']

0        -4542.0
1        -4542.0
2        -1134.0
3        -3051.0
4        -3051.0
           ...  
438752   -3331.0
438753   -5941.0
438754    -880.0
438755    -885.0
438756    -783.0
Name: DAYS_EMPLOYED, Length: 365386, dtype: float64

In [22]:
df_main.loc[df_main['DAYS_EMPLOYED'] >= 0, 'DAYS_EMPLOYED'] = 0

In [23]:
df_main.loc[df_main['DAYS_EMPLOYED'] < 0, 'DAYS_EMPLOYED'] = 1

Merge

In [25]:
df_paid = df_paid[['ID','STATUS']]

In [28]:
df_main = df_main[['Main_ID','CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AGE', 'DAYS_EMPLOYED','CNT_FAM_MEMBERS']]

In [30]:
df_final = df_main.merge(df_paid, left_on = 'Main_ID', right_on = 'ID')

In [31]:
corr = df_main.corr()
corr.style.background_gradient()

,Main_ID,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AGE,DAYS_EMPLOYED,CNT_FAM_MEMBERS
Main_ID,1.000000,0.000623,0.041532,-0.004956,0.011031,0.005245,0.002349,-0.001911
FLAG_OWN_CAR,0.000623,1.000000,-0.002125,0.110793,0.181654,-0.159845,0.159823,0.151278
FLAG_OWN_REALTY,0.041532,-0.002125,1.000000,0.001630,0.024127,0.105469,-0.062971,0.007480
CNT_CHILDREN,-0.004956,0.110793,0.001630,1.000000,0.017719,-0.342874,0.234769,0.858879
AMT_INCOME_TOTAL,0.011031,0.181654,0.024127,0.017719,1.000000,-0.052854,0.136840,0.010696
AGE,0.005245,-0.159845,0.105469,-0.342874,-0.052854,1.000000,-0.612307,-0.302002
DAYS_EMPLOYED,0.002349,0.159823,-0.062971,0.234769,0.136840,-0.612307,1.000000,0.228383
CNT_FAM_MEMBERS,-0.001911,0.151278,0.007480,0.858879,0.010696,-0.302002,0.228383,1.000000


In [32]:
df_final.head()

,Main_ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AGE,DAYS_EMPLOYED,CNT_FAM_MEMBERS,ID,STATUS
0,5008804,M,True,True,0.0,427500.0,33.0,1.0,2.0,5008804,C
1,5008804,M,True,True,0.0,427500.0,33.0,1.0,2.0,5008804,C
2,5008804,M,True,True,0.0,427500.0,33.0,1.0,2.0,5008804,C
3,5008804,M,True,True,0.0,427500.0,33.0,1.0,2.0,5008804,C
4,5008804,M,True,True,0.0,427500.0,33.0,1.0,2.0,5008804,C


In [36]:
df_final['TARGET'] = df_final['STATUS'].isin(['C', 'X', 0])

In [ ]:
df_final['TARGET'].uique

In [34]:
#define target value
df_final.to_csv('final_df.csv', sep=';')